<a href="https://colab.research.google.com/github/riccardomarin/EG22_Tutorial_Spectral_Geometry/blob/main/forward/02_Shape_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we will replicate some of our consideration for Graphs on 3D surfaces

In [ ]:
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/data/tr_reg_090.off
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/data/tr_reg_043.off
!wget https://github.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/raw/main/data/pose.mat
!wget https://github.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/raw/main/data/style.mat

!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/utils/utils_mesh.py
!wget https://raw.githubusercontent.com/riccardomarin/EG22_Tutorial_Spectral_Geometry/main/utils/utils_spectral.py

!pip install plyfile

In [ ]:
import os 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import scipy.io as sio
import scipy.sparse.linalg
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix

import numpy as np
import pandas as pd
import torch  
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 

from utils_spectral import LB_FEM_sparse, EigendecompositionSparse, LB_cotan, Eigendecomposition
from utils_mesh import load_off
from utils_mesh import plot_colormap

Load a 3D model as a triangular mesh

In [ ]:
# In this case the mesh is saved in a matlab-like file
shape_path1 = './pose.mat'
dtype = 'float32'

x = sio.loadmat(shape_path1)

# Converting the dictionary in convenient variables, and moving them on the GPU
vertices = torch.from_numpy(x['M']['VERT'][0,0].astype(dtype)).cuda()
triv = torch.from_numpy(x['M']['TRIV'][0,0].astype('long')).cuda()-1

We provide two ways to compute the LBO in the supporting utilities: a sparse one and a dense one. They produce the same results, but for different applications you may prefer one instead of the other.

In [ ]:
# Number of eigenvalues we would compute
k = 90

# Sparse LBO
L_sym_sp, A_sp, Ainv_sp = LB_FEM_sparse(vertices,triv.long())

evecs_sp, evals_sp = EigendecompositionSparse(L_sym_sp.values(), L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs_sp = evecs_sp * Ainv_sp[:,None]

# Dense LBO
W, A = LB_cotan(vertices ,triv.long())
Ainv = A.rsqrt()
L_sym = torch.mul(W, Ainv[None, :] * Ainv[:, None])  # <- INEFFICIENCY

evecs, evals = Eigendecomposition(L_sym, torch.tensor(k), torch.tensor(L_sym.shape[-1]))
evecs = evecs * Ainv[:,None]

Comparing eigenvectors obtained by the two eigendecompositions

In [ ]:
p = plot_colormap( [vertices]*3, [triv]*3,[evecs[:,i+1] for i in range(3)] )
p.show()

p = plot_colormap( [vertices]*3, [triv]*3,[evecs_sp[:,i+1] for i in range(3)] )
p.show()

Visualizing the spectra

In [ ]:
plt.plot(evals.detach().cpu(), 'r', linewidth=4)    #Dense method
plt.plot(evals_sp.detach().cpu(),'k--',linewidth=4) #Sparse method

Coordinates Low Pass

In [ ]:
k = 10

evecs_trim = evecs[:,0:k]

# Remember: the inner product is defined with the Areas!
v = evecs[:,0:k] @ evecs_trim.T @ (A[:,None] * vertices)

# Low pass representation compared to the original mesh
p = plot_colormap([v, vertices], [triv]*2,[np.ones(v.shape[0]),np.ones(v.shape[0])] )
p.show()

We can also try on a more articulated shape.

In [ ]:
# Loading an humanoid shape
with open("tr_reg_090.off") as f:
  v, f = load_off(f)

vertices = torch.from_numpy(v.astype(dtype)).cuda()*1
triv = torch.from_numpy(np.asarray(f).astype('long')).cuda()

k = 50

# Computing the eigenvectors
L_sym_sp, A_sp, Ainv_sp = LB_FEM_sparse(vertices,triv.long())
evecs, evals = EigendecompositionSparse(L_sym_sp.values(),L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs = evecs * Ainv_sp[:,None]

# Low
evecs_trim = evecs[:,0:k]
v = evecs_trim @ (evecs_trim.T * A_sp[None,:]) @ vertices

p = plot_colormap([v]*3, [triv]*3,[np.ones(v.shape[0])] )
p.show()

# Spectral Clustering (Segmentation)

As a first application, we can cluster the surface using the spectral embedding of the shapes.

The steps are:

- Choosing the number of clusters
- Running KMeans on the eigenvectors
- Visualizing the clusters

In [ ]:
# Number of cluster
n_c = 6

#KMeans
kmeans = KMeans(n_clusters=n_c, random_state=1).fit(evecs[:,1:n_c].detach().cpu())

# Visualization
p = plot_colormap([vertices]*3, [triv]*3,[kmeans.labels_] )
p.show()

Can we expect that for two different shapes the result will be similar? The first eigenfunctions are pretty stable among objects of the same class, so they are good features to cluster the points.

In [ ]:
with open("tr_reg_043.off") as f:
  v, f = load_off(f)

vertices2 = torch.from_numpy(v.astype(dtype)).cuda()*1
triv2 = torch.from_numpy(np.asarray(f).astype('long')).cuda()
L_sym_sp, A_sp, Ainv_sp = LB_FEM_sparse(vertices2,triv2.long())
evecs2, evals2 = EigendecompositionSparse(L_sym_sp.values(),L_sym_sp.indices(), torch.tensor(k), torch.tensor(L_sym_sp.shape[-1]))
evecs2 = evecs2 * Ainv_sp[:,None]

p = plot_colormap([vertices,vertices,vertices,vertices], [triv,triv,triv,triv],
                  [evecs[:,1], evecs[:,2], evecs[:,3],evecs[:,4]] )
p.show()

p = plot_colormap([vertices2,vertices2,vertices2,vertices2], [triv2,triv2,triv2,triv2],
                  [evecs2[:,1], evecs2[:,2], evecs2[:,3],evecs2[:,4]] )
p.show()

Now we can compute the cluster also for the second shape.

In [ ]:
n_c = 6
kmeans2 = KMeans(n_clusters=n_c, random_state=1).fit(evecs2[:,1:n_c].detach().cpu())

p = plot_colormap([vertices,vertices2], [triv,triv2]*3,[kmeans.labels_, kmeans2.labels_] )
p.show()

Using higher frequencies, the process becomes more unstable. Plotting the eigenvalues of the two shapes gives us a good intuition of this fact.

In [ ]:
plt.plot(evals.detach().cpu(), 'b', linewidth=4)
plt.plot(evals2.detach().cpu(),'r',linewidth=4)